In [ ]:
    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Big Mart Data Analysis Using Machine Learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('../input/big-mart-sales/train_v9rqX0R.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df['Item_Fat_Content'].unique()

In [ ]:
train_df['Outlet_Establishment_Year'].unique()

In [ ]:
train_df['Outlet_Age']=2018-train_df['Outlet_Establishment_Year']
train_df.head()

In [ ]:
train_df['Outlet_Size'].unique()

In [ ]:
train_df.describe()

In [ ]:
train_df['Item_Fat_Content'].value_counts()

In [ ]:
train_df['Outlet_Size'].mode()[0]

In [ ]:
train_df['Outlet_Size']=train_df['Outlet_Size'].fillna(train_df['Outlet_Size'].mode()[0])

In [ ]:
train_df['Item_Weight']=train_df['Item_Weight'].fillna(train_df['Item_Weight'].mean())

In [ ]:
train_df['Item_Visibility'].hist(bins=20)

In [ ]:
train_df.hist(figsize=(15,12))

In [ ]:
Q1 = train_df['Item_Visibility'].quantile(0.25)
Q3 = train_df['Item_Visibility'].quantile(0.75)
IQR = Q3 - Q1
filt_train = train_df.query('(@Q1 - 1.5*@IQR) <= Item_Visibility <=(@Q3 + 1.5 * @IQR)')

In [ ]:
filt_train

In [ ]:
filt_train.shape, train_df.shape

In [ ]:
train_df = filt_train
train_df.shape

In [ ]:
train_df['Item_Visibility_bins'] = pd.cut(train_df['Item_Visibility'],[0.000, 0.065, 0.13,0.2],
                                        labels=['Low Viz','Viz','High Viz'])

In [ ]:
train_df['Item_Visibility_bins'].value_counts()

In [ ]:
train_df['Item_Visibility_bins'] = train_df['Item_Visibility_bins'].replace(np.nan, 'Low Viz',regex=True)

In [ ]:
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].replace(['Low fat','LF'],'Low Fat')

In [ ]:
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].replace('reg','Regular')

In [ ]:
train_df.head()

In [ ]:
le = LabelEncoder()

In [ ]:
train_df['Item_Fat_Content'].unique()

In [ ]:
train_df['Item_Fat_Content']=le.fit_transform(train_df['Item_Fat_Content'])

In [ ]:
train_df['Item_Visibility_bins']=le.fit_transform(train_df['Item_Visibility_bins'])

In [ ]:
train_df['Outlet_Size']=le.fit_transform(train_df['Outlet_Size'])

In [ ]:
train_df['Outlet_Location_Type']=le.fit_transform(train_df['Outlet_Location_Type'])

In [ ]:
dummy = pd.get_dummies(train_df['Outlet_Type'])
dummy.head()

In [ ]:
train_df =pd.concat([train_df,dummy],axis=1)

In [ ]:
train_df.dtypes

In [ ]:
train_df=train_df.drop(['Item_Identifier','Item_Type','Outlet_Identifier','Outlet_Type',
                      'Outlet_Establishment_Year' ],axis=1)

In [ ]:
train_df.columns

In [ ]:
train_df.head()

### Linear Regression

* Here We Will Apply Linear Regression to this Dataset

In [ ]:
x = train_df.drop('Item_Outlet_Sales',axis=1)
y = train_df.Item_Outlet_Sales

In [ ]:
test = pd.read_csv('../input/big-mart11/Test_u94Q5KV.csv')
test['Outlet_Size']=test['Outlet_Size'].fillna('Medium')

In [ ]:
test['Item_Visibility_bins']=pd.cut(test['Item_Visibility'],[0.000 ,0.065,0.13,0.2],
                                   labels=['Low Viz','Viz','High Viz'])

In [ ]:
test['Item_Weight']=test['Item_Weight'].fillna(test['Item_Weight'].mean())

In [ ]:
test['Item_Visibility_bins']=test['Item_Visibility_bins'].fillna('Low Viz')
test['Item_Visibility_bins'].head()

In [ ]:
test['Item_Fat_Content']=le.fit_transform(test['Item_Fat_Content'])

In [ ]:
test['Item_Visibility_bins']=le.fit_transform(test['Item_Visibility_bins'])

In [ ]:
test['Outlet_Size']=le.fit_transform(test['Outlet_Size'])

In [ ]:
test['Outlet_Location_Type']=le.fit_transform(test['Outlet_Location_Type'])

In [ ]:
test['Outlet_Age']=le.fit_transform(test['Outlet_Establishment_Year'])

In [ ]:
dummy = pd.get_dummies(test['Outlet_Type'])
test = pd.concat([test,dummy],axis=1)

In [ ]:
x_test = test.drop(['Item_Identifier','Item_Type','Outlet_Type','Outlet_Establishment_Year'],axis=1)

In [ ]:
x.columns,x_test.columns

## Predictive Modeling

In [ ]:
from sklearn import model_selection
xtrain,xtest,ytrain,ytest=model_selection.train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
lin = LinearRegression()

In [ ]:
lin.fit(xtrain,ytrain)
print(lin.coef_)
lin.intercept_

In [ ]:
predictions = lin.predict(xtest)
print(sqrt(mean_squared_error(ytest,predictions)))

In [ ]:
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0.001,normalize=True)
ridgereg.fit(xtrain,ytrain)

print(sqrt(mean_squared_error(ytrain,ridgereg.predict(xtrain))))
print(sqrt(mean_squared_error(ytest,ridgereg.predict(xtest))))
print('R2 Value/Coefficient of Determination:{}'.format(ridgereg.score(xtest,ytest)))

In [ ]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.001,normalize=True)
lassoreg.fit(xtrain,ytrain)

print(sqrt(mean_squared_error(ytrain,lassoreg.predict(xtrain))))
print(sqrt(mean_squared_error(ytest,lassoreg.predict(xtest))))
print('R2 Value/Coefficient of Determination:{}'.format(lassoreg.score(xtest,ytest)))

In [ ]:
from sklearn.linear_model import ElasticNet
Elas = ElasticNet(alpha=0.001,normalize=True)
Elas.fit(xtrain,ytrain)

print(sqrt(mean_squared_error(ytrain,Elas.predict(xtrain))))
print(sqrt(mean_squared_error(ytest,Elas.predict(xtest))))
print('R2 Value/Coefficient of Determination:{}'.format(Elas.score(xtest,ytest)))